Training is hard, optimization is totally different story

computationally its too hard
<a href="https://ibb.co/1GZVm5s"><img src="https://i.ibb.co/k2KnSf3/image.png" alt="image" border="0"></a>


u start in bad place, u dead
## To solve this
- we can do some optimization technique to traverse the local minimas to global minima
- make nn which has good landscape, which are more smoother
## $n$ is the learning rate which also needs proper tuning
- if its too small , it can get stuck in local minimas/ false local minima
- if its too big, it overshoots the global minima
- so we need to tune it properly, i.e. stable learning rate, ( which can overshoot local minima, but finds global minima )

# billions of dimension and each dimension of W from (-inf, inf), so its not restricted and too hard to work bith , in BIG Models

## soluton for learning rates
- hit and trial
- design a adaptive learning rate that adapts to the landscape

Adaptive Learning Rates
- Learning rates are no longer fixed
- Can be made larger or smaller depending on:
- how large gradient is
- how fast learning is happening
- size of particular weights
- etc...

## Gradient descent algorithms
- Adagrad
- RMSprop
- Adam
- Adadelta
- SGD(Stochastic Gradient Descent)
- etc...


In [ ]:
import tensorflow as tf

tf.keras.optimizers.SGD
tf.keras.optimizers.Adam
tf.keras.optimizers.RMSprop
tf.keras.optimizers.Adagrad
tf.keras.optimizers.Adadelta
tf.keras.optimizers.Adamax
tf.keras.optimizers.Nadam
tf.keras.optimizers.Ftrl
tf.keras.optimizers.Optimizer#(for class inheritance)

Reference
Kiefer & Wolfowitz. "Stochastic Estimation of the
Maximum of a Regression Function." 1952.<br>
Kingma et al. "Adam: A Method for Stochastic
Optimization." 2014.<br>
Zeiler et al."ADADELTA: An Adaptive Learning Rate
Method." 2012.<br>
Duchi et al. "Adaptive Subgradient Methods for Online
Learning and Stochastic Optimization." 20 1 1.

Additional details: http://ruderio/optimizing-gradient-descent/

In [32]:
import tensorflow as tf
import numpy as np

# Generate some sample data
np.random.seed(42)
X = np.random.rand(1000, 2)
y = (X[:, 0] + X[:, 1] > 1).astype(float)
X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)

# Create a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Define optimizer
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# Define loss function
def compute_loss(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true, y_pred))

# Training loop
num_epochs = 100
batch_size = 32
num_batches = len(X) // batch_size

for epoch in range(num_epochs):
    total_loss = 0
    
    # Create batches
    indices = tf.random.shuffle(tf.range(len(X)))
    X_shuffled = tf.gather(X, indices)# gather gathers data from the tensor X based on the indices
    y_shuffled = tf.gather(y, indices)
    
    for batch in range(num_batches):
        # Get batch data
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_shuffled[start_idx:end_idx]
        y_batch = y_shuffled[start_idx:end_idx]
        y_batch = tf.reshape(y_batch, (len(y_batch), 1))
        
        # Training step
        with tf.GradientTape() as tape:
            # Forward pass
            predictions = model(X_batch)
            # Compute loss
            loss = compute_loss(y_batch, predictions)
        
        # Compute gradients
        gradients = tape.gradient(loss, model.trainable_variables)
        # Update weights
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        total_loss += loss
    
    # Print progress every 10 epochs
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss/num_batches:.4f}")

# Test the model
X = np.random.rand(1000, 2)
y = (X[:, 0] + X[:, 1] > 1).astype(float)
X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)
test_predictions = model(X)
accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(test_predictions), y), tf.float32))
print(f"\nFinal accuracy: {accuracy:.4f}")

Epoch 10/100, Loss: 0.1604
Epoch 20/100, Loss: 0.0762
Epoch 30/100, Loss: 0.0602
Epoch 40/100, Loss: 0.0575
Epoch 50/100, Loss: 0.0359
Epoch 60/100, Loss: 0.0502
Epoch 70/100, Loss: 0.0373
Epoch 80/100, Loss: 0.0337
Epoch 90/100, Loss: 0.0385
Epoch 100/100, Loss: 0.0431

Final accuracy: 0.5000


In [29]:
import tensorflow as tf
import numpy as np

# Generate some sample data
np.random.seed(42)
X = np.random.rand(1000, 2)
y = (X[:, 0] + X[:, 1] > 1).astype(float)

# Reshape y to have shape (n_samples, 1)
y = y.reshape(-1, 1)

X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)

# Create a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

# Define loss function
def compute_loss(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true, y_pred))

# Training loop
num_epochs = 100
batch_size = 100
num_batches = len(X) // batch_size

for epoch in range(num_epochs):
    total_loss = 0
    
    # Create batches
    indices = tf.random.shuffle(tf.range(len(X)))
    X_shuffled = tf.gather(X, indices)
    y_shuffled = tf.gather(y, indices)
    
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_shuffled[start_idx:end_idx]
        y_batch = y_shuffled[start_idx:end_idx]
        
        with tf.GradientTape() as tape:
            predictions = model(X_batch)
            loss = compute_loss(y_batch, predictions)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        total_loss += loss
    
    if (epoch + 1) % 10 == 0:
        print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {total_loss/num_batches:.4f}")

# Test the model
test_predictions = model(X)
binary_predictions = tf.cast(test_predictions > 0.5, tf.float32)
accuracy = tf.reduce_mean(tf.cast(tf.equal(binary_predictions, y), tf.float32))
print(f"\nFinal accuracy: {accuracy:.4f}")

# Print some additional metrics
print("\nDetailed metrics:")
from sklearn.metrics import classification_report
y_pred = binary_predictions.numpy()
y_true = y.numpy()
print(classification_report(y_true, y_pred))

Epoch 10/100, Loss: 0.0704
Epoch 20/100, Loss: 0.0427
Epoch 30/100, Loss: 0.0404
Epoch 40/100, Loss: 0.0331
Epoch 50/100, Loss: 0.0470
Epoch 60/100, Loss: 0.0222
Epoch 70/100, Loss: 0.0203
Epoch 80/100, Loss: 0.0211
Epoch 90/100, Loss: 0.0272
Epoch 100/100, Loss: 0.0204

Final accuracy: 0.9970

Detailed metrics:
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       496
         1.0       1.00      1.00      1.00       504

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000



In [21]:
import tensorflow as tf
import numpy as np

# Generate some sample data - with clearer separation
np.random.seed(42)
X = np.random.rand(1000, 2) * 2 - 1  # Values between -1 and 1
y = (X[:, 0] + X[:, 1] > 0).astype(float)  # Clear decision boundary
X = tf.convert_to_tensor(X, dtype=tf.float32)
y = tf.convert_to_tensor(y, dtype=tf.float32)
y = tf.reshape(y, (-1, 1))  # Reshape to match model output

# Create a simple model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(16, activation='relu', input_shape=(2,)),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Define optimizer with a proper learning rate
optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

# Define loss function
def compute_loss(y_true, y_pred):
    return tf.reduce_mean(tf.keras.losses.binary_crossentropy(y_true, y_pred))

# Split data into train and validation sets
train_size = int(0.8 * len(X))
X_train, X_val = X[:train_size], X[train_size:]
y_train, y_val = y[:train_size], y[train_size:]

# Training loop
num_epochs = 100
batch_size = 32
num_batches = len(X_train) // batch_size

for epoch in range(num_epochs):
    total_loss = 0
    
    # Create batches
    indices = tf.random.shuffle(tf.range(len(X_train)))
    X_shuffled = tf.gather(X_train, indices)
    y_shuffled = tf.gather(y_train, indices)
    
    for batch in range(num_batches):
        start_idx = batch * batch_size
        end_idx = start_idx + batch_size
        X_batch = X_shuffled[start_idx:end_idx]
        y_batch = y_shuffled[start_idx:end_idx]
        
        with tf.GradientTape() as tape:
            predictions = model(X_batch)
            loss = compute_loss(y_batch, predictions)
        
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
        total_loss += loss
    
    # Evaluate on validation set
    if (epoch + 1) % 10 == 0:
        val_predictions = model(X_val)
        val_loss = compute_loss(y_val, val_predictions)
        val_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(val_predictions), y_val), tf.float32))
        print(f"Epoch {epoch + 1}/{num_epochs}")
        print(f"Training Loss: {total_loss/num_batches:.4f}")
        print(f"Validation Loss: {val_loss:.4f}")
        print(f"Validation Accuracy: {val_accuracy:.4f}\n")

# Final evaluation
test_predictions = model(X_val)
final_accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.round(test_predictions), y_val), tf.float32))
print(f"Final test accuracy: {final_accuracy:.4f}")

# Visualize some predictions
print("\nSample predictions vs actual values:")
for i in range(5):
    pred = model(X_val[i:i+1])
    print(f"Input: [{X_val[i,0]:.2f}, {X_val[i,1]:.2f}]")
    print(f"Predicted: {pred.numpy()[0,0]:.4f}, Actual: {y_val[i,0]:.0f}")

/home/mtech/2021/dibyo/.anaconda/envs/CPU2/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 10/100
Training Loss: 0.0772
Validation Loss: 0.0612
Validation Accuracy: 1.0000

Epoch 20/100
Training Loss: 0.0448
Validation Loss: 0.0361
Validation Accuracy: 1.0000

Epoch 30/100
Training Loss: 0.0345
Validation Loss: 0.0332
Validation Accuracy: 0.9900

Epoch 40/100
Training Loss: 0.0290
Validation Loss: 0.0257
Validation Accuracy: 0.9950



KeyboardInterrupt: 